<a href="https://colab.research.google.com/github/MSadimba/milvusdb-test/blob/main/milvusdbtest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate sentencepiece bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

rai_classifier = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4,
    temperature=0.0
)

In [ ]:
def classify_document_rai(text: str) -> bool:
    """
    Free document-level classifier using LLaMA-3.1 8B.
    Returns True if the paper is genuinely Responsible AI.
    """
    prompt = f"""
You are a strict Responsible AI classifier.

Classify this paper as either:
RAI_TRUE  – truly about responsible AI, AI safety, fairness, alignment, governance,
            transparency, accountability, risk management, model oversight,
            ethical AI evaluation, or compliance.

RAI_FALSE – political commentary, activism, sociology, unrelated philosophy,
            general CS/ML not about safety, hype, fiction, marketing,
            or irrelevant content.

Return ONLY: RAI_TRUE or RAI_FALSE.

Paper text (sample):
{text[:4000]}
"""

    out = rai_classifier(prompt)[0]["generated_text"]
    out = out.strip().upper()

    return "RAI_TRUE" in out